In [3]:
from libs.util.secret import PINECONE_API_KEY, ANTHROPIC_API_KEY

API_KEY = ANTHROPIC_API_KEY

# 주어진 텍스트를 임베딩 모델을 활용하여 벡터로 변환

In [11]:
from pinecone import Pinecone

# Pinecone 클라이언트 초기화
pc = Pinecone(api_key=PINECONE_API_KEY)

# 데이터 정의
data = [
    {"id": "vec1", "text": "자동차 보험 청구를 위해서는 사고 발생 보고서, 차량 수리 견적서, 보험 증서 사본이 필요하다."},
    {"id": "vec2", "text": "건강 보험 청구를 위해서는 병원 진단서, 치료비 영수증, 보험 증서 사본이 필요하다."},
    {"id": "vec3", "text": "화재 보험 청구를 위해서는 화재 감식 보고서, 피해 내역서, 보험 증서 사본이 필요하다."},
]

# 텍스트 데이터를 Pinecone이 색인할 수 있는 형태의 벡터로 변환
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in data],
    parameters={
        # 모델에게 입력 텍스트가 어떤 형태인지 알려주는 것
        # "query": 검색 질의
        # "passage": 문서의 본문처럼 긴 문단 또는 설명형 텍스트
        "input_type": "passage",

        # 입력 길이가 모델의 최대 토큰 수를 초과할 경우, 어떤 식으로 잘라낼지를 결정
        # "START": 앞부분을 잘라낸다. (뒤쪽 보존)
        # "END": 뒷부분을 잘라낸다. (앞쪽 보존)
        "truncate": "END",
    },
)

print(embeddings)
print(f'dimension: {len(embeddings.data[0]["values"])}')

EmbeddingsList(
  model='multilingual-e5-large',
  vector_type='dense',
  data=[
    {'vector_type': dense, 'values': [-0.0033245086669921875, -0.042236328125, ..., -0.04840087890625, -0.007171630859375]},
    {'vector_type': dense, 'values': [0.0158233642578125, -0.040771484375, ..., -0.039337158203125, -0.0187225341796875]},
    {'vector_type': dense, 'values': [-0.01415252685546875, -0.0187225341796875, ..., -0.030853271484375, 0.00612640380859375]}
  ],
  usage={'total_tokens': 89}
)
dimension: 1024


# 벡터를 저장할 인덱스 생성

In [7]:
from pinecone import ServerlessSpec

# 인덱스 생성
index_name = "example-index"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        # 임베딩 차원
        # 앞에서 사용한 임베딩 모델(multilingual-e5-large)의 출력 벡터 크기와 정확히 일치해야 한다.
        dimension=1024,
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        ),
    )

# 벡터를 인덱스에 저장

In [12]:
# 벡터를 저장할 인덱스 가져오기
index = pc.Index("example-index")

# 인덱스에 저장하기위해 records 생성
# id: 필수 값, values: 임베딩 벡터, metadata: 텍스트
records = []
for d, e in zip(data, embeddings):
    records.append({
        "id": d['id'],
        "values": e['values'],
        # 메타데이터에 원본 텍스트 값을 함께 저장하는것이 일반적
        "metadata": {'text': d['text']}
    })

# records를 index에 저장하기
index.upsert(
    vectors=records,
    namespace="example-namespace"
)

{'upserted_count': 3}

# 입력 쿼리 벡터화

In [13]:
query = "자동차 보험 청구에 필요한 서류가 뭐야?"

x = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    },
)

print(x)

EmbeddingsList(
  model='multilingual-e5-large',
  vector_type='dense',
  data=[
    {'vector_type': dense, 'values': [-0.0186309814453125, -0.01238250732421875, ..., -0.05035400390625, -0.0080413818359375]}
  ],
  usage={'total_tokens': 17}
)


# 유사도가 높은 문서 검색
- 빠른 검색(Recall)
- 쿼리를 벡터로 변환후 벡터 유사도로 검색 (1차 검색 단계)
- 검색 속도 빠름, 정확도 비교적 낮음
- 쿼리 문서를 개별적으로 임베딩 (Bi-Encoder)

In [14]:
# 인덱스 가져오기
index = pc.Index("example-index")

# 유사 문서 검색
results = index.query(
    namespace="example-namespace",
    vector=x[0].values, # 입력 쿼리가 변환된 벡터
    top_k=3, # 유사도 점수 기준으로 상위 3개 검색
    include_values=False, # 응답에 벡터 값을 포함하지 않는다
    include_metadata=True # 응답에 메타 데이터를 포함한다
)

# 검색 문서 출력
print(results)

{'matches': [{'id': 'vec1',
              'metadata': {'text': '자동차 보험 청구를 위해서는 사고 발생 보고서, 차량 수리 견적서, 보험 '
                                   '증서 사본이 필요하다.'},
              'score': 0.901361167,
              'values': []},
             {'id': 'vec2',
              'metadata': {'text': '건강 보험 청구를 위해서는 병원 진단서, 치료비 영수증, 보험 증서 사본이 '
                                   '필요하다.'},
              'score': 0.864458621,
              'values': []},
             {'id': 'vec3',
              'metadata': {'text': '화재 보험 청구를 위해서는 화재 감식 보고서, 피해 내역서, 보험 증서 '
                                   '사본이 필요하다.'},
              'score': 0.856482446,
              'values': []}],
 'namespace': 'example-namespace',
 'usage': {'read_units': 6}}


# 유사도 높은 문서 검색 이후 문서 재정렬 (Re-ranking)
- 검색 결과의 정밀도 향상 (Precision)
- 검색된 문서를 다시 비교해 정렬 (2차 필터링)
- 검색 속도 느림, 정확도 비교적 높음
- 쿼리와 문서를 함께 입력 (Cross-Encoder)

In [15]:
result = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=query,
    documents=[{"id": match["id"], "text": match["metadata"]["text"]} for match in results["matches"]],
    top_n=3,
    return_documents=True,
    parameters={
        "truncate": "END"
    }
)

# 임베딩 모델에서는 문서간 점수 차이가 크지 않았지만,
# 재순위화 모델에서는 높은 문서의 점수만 크다.
print(result)

RerankResult(
  model='bge-reranker-v2-m3',
  data=[{
    index=0,
    score=0.9982248,
    document={
        id='vec1',
        text='자동차 보험 청구를 위해서는 사고 발생 보고서, 차량 수리 견적서, 보험 증서 사본이 필요하다.'
    }
  },{
    index=2,
    score=0.16899405,
    document={
        id='vec3',
        text='화재 보험 청구를 위해서는 화재 감식 보고서, 피해 내역서, 보험 증서 사본이 필요하다.'
    }
  },{
    index=1,
    score=0.15947905,
    document={
        id='vec2',
        text='건강 보험 청구를 위해서는 병원 진단서, 치료비 영수증, 보험 증서 사본이 필요하다.'
    }
  }],
  usage={'rerank_units': 1}
)


---

# RecursiveCharacterTextSplitter를 이용한 문자 기반 청킹 예시

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

sample_text = """자동차 보험 청구를 위해서는 사고 발생 보고서, 차량 수리 견적서, 보험증서 사본이 필요하다.
건강 보험 청구를 위해서는 병원 진단서, 치료비 영수증, 보험 증서 사본이 필요하다.
화재 보험 청구를 위해서는 화재 감식 보고서, 피해 내역서, 보험 증서 사본이 필요하다."""


overlap_splitter = RecursiveCharacterTextSplitter(
    chunk_size=30, # 청크 크기
    chunk_overlap=10,  # 오버랩 설정
    length_function=len # 분할 기준 길이 측정 함수 (len=문자열 길이)
)
overlap_chunks = overlap_splitter.split_text(sample_text)

print(len(overlap_chunks))
print(overlap_chunks)

7
['자동차 보험 청구를 위해서는 사고 발생 보고서, 차량', '보고서, 차량 수리 견적서, 보험증서 사본이', '보험증서 사본이 필요하다.', '건강 보험 청구를 위해서는 병원 진단서, 치료비', '진단서, 치료비 영수증, 보험 증서 사본이 필요하다.', '화재 보험 청구를 위해서는 화재 감식 보고서, 피해', '보고서, 피해 내역서, 보험 증서 사본이 필요하다.']


# MarkdownHeaderTextSplitter를 이용한 마크다운 문서 청킹 예시

In [17]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

sample_text = """
# 제목

이것은 문서의 소개 섹션입니다. 문서의 개요를 제공합니다.

## 섹션 1

이 섹션에서는 첫 번째 주제를 다룹니다. 관련된 세부 정보가 포함되어 있습니다.

## 섹션 2

이 섹션에서는 두 번째 주제를 논의합니다. 여기에서 더 많은 세부 사항이 설명됩니다.
"""

md_splitter = MarkdownHeaderTextSplitter(
    # 문서를 분할할 헤더 레벨과 해당 레벨의 이름 정의
    headers_to_split_on=[("#", "Header 1"), ("##", "Header 2")]
)
md_chunks = md_splitter.split_text(sample_text)

print(len(md_chunks))
print(md_chunks)

3
[Document(metadata={'Header 1': '제목'}, page_content='이것은 문서의 소개 섹션입니다. 문서의 개요를 제공합니다.'), Document(metadata={'Header 1': '제목', 'Header 2': '섹션 1'}, page_content='이 섹션에서는 첫 번째 주제를 다룹니다. 관련된 세부 정보가 포함되어 있습니다.'), Document(metadata={'Header 1': '제목', 'Header 2': '섹션 2'}, page_content='이 섹션에서는 두 번째 주제를 논의합니다. 여기에서 더 많은 세부 사항이 설명됩니다.')]


# 의미 기반 커스텀 문서 청킹 로직 구현

In [21]:
sample_text = """자동차 보험 청구를 위해서는 사고 발생 보고서, 차량 수리 견적서, 보험증서 사본이 필요하다.
사고 발생 보고서는 사고의 경위와 책임 여부를 확인하는 중요한 자료다.
차량 수리 견적서는 수리 비용을 산정하는 근거가 되며, 보험증서 사본은 가입한 보험 상품의 보장 범위를 확인하는 데 사용된다.
화재 보험 청구를 위해서는 화재 감식 보고서, 피해 내역서, 보험 증서 사본이 필요하다.
화재 감식 보고서는 화재의 원인과 피해 규모를 분석한 공식 문서다.
피해 내역서는 손실된 재산의 종류와 피해 금액을 구체적으로 정리한 자료이며, 보험 증서 사본은 보장 범위와 보상 한도를 확인하는 데 사용된다."""

# 1. 문장을 줄바꿈("\n")을 기준으로 분리하여 리스트로 저장
sentences = sample_text.split("\n")

# 2. 유사한 문장을 합치기 위한 빈 리스트 생성
chunks = []

# 3. 첫 번째 문장을 기준으로 설정
before_sentence = sentences[0]

# 4. 나머지 문장들과 비교하여 유사도를 평가
for sentence in sentences[1:]:
    # Reranker 모델을 사용하여 두 문장의 유사도 점수 계산
    result = pc.inference.rerank(
        model="bge-reranker-v2-m3",
        query=before_sentence,  # 기준 문장
        documents=[{"id": "1", "text": sentence}],  # 비교할 문장
        top_n=1,
        parameters={"truncate": "END"}  # 긴 문장일 경우 뒤쪽을 자름
    )

    # 계산된 유사도 점수 가져오기
    score = result.data[0].score

    # 5. 유사도 점수가 0.9 이상이면 문장을 결합
    if score >= 0.9:
        before_sentence = "\n".join([before_sentence, sentence])
    else:
        # 현재까지 합쳐진 문장을 chunks 리스트에 저장
        chunks.append(before_sentence)
        # 새로운 기준 문장을 설정
        before_sentence = sentence

# 마지막 기준 문장도 chunks 리스트에 추가
chunks.append(before_sentence)

# 6. 최종 그룹화된 문장 출력
for idx, chunk in enumerate(chunks, 1):
    print(f"\n[그룹 {idx}]")
    print(chunk)


[그룹 1]
자동차 보험 청구를 위해서는 사고 발생 보고서, 차량 수리 견적서, 보험증서 사본이 필요하다.
사고 발생 보고서는 사고의 경위와 책임 여부를 확인하는 중요한 자료다.
차량 수리 견적서는 수리 비용을 산정하는 근거가 되며, 보험증서 사본은 가입한 보험 상품의 보장 범위를 확인하는 데 사용된다.

[그룹 2]
화재 보험 청구를 위해서는 화재 감식 보고서, 피해 내역서, 보험 증서 사본이 필요하다.
화재 감식 보고서는 화재의 원인과 피해 규모를 분석한 공식 문서다.
피해 내역서는 손실된 재산의 종류와 피해 금액을 구체적으로 정리한 자료이며, 보험 증서 사본은 보장 범위와 보상 한도를 확인하는 데 사용된다.


---

In [24]:
!pip install -q pypdf


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


# PDF 파일 읽기

In [25]:
from langchain_community.document_loaders import PyPDFLoader

# pdf 파일 경로 정의
pdf_filepaths = [
    '~/PycharmProjects/yuncheol-llmops-demo/dataset/주택화재보험_상품요약서.pdf',
    '~/PycharmProjects/yuncheol-llmops-demo/dataset/자동차보험_상품요약서.pdf',
    '~/PycharmProjects/yuncheol-llmops-demo/dataset/실손의료비보험_상품요약서.pdf',
]

# 배열 순회하면서 pdf 파일 경로 읽기
documents = []
for pdf_filepath in pdf_filepaths:
    # PDF 페이지별로 읽기
    loader = PyPDFLoader(pdf_filepath)
    pages = loader.load()
    # 하나의 문서로 합치기
    document = "".join(page.page_content for page in pages)
    documents.append(document)

print(len(documents))
print(documents[:1])

3
['KB주택화재보험 상품요약서1.가입자격 제한 등 상품의 특이사항가.가입자격 제한주택으로만 쓰이는 건물 중 다음의 것 및 그 수용가재에 대하여 적용합니다.1)단독주택2)주택의 부속건물로서 가재만을 수용하는데 쓰이는 것3)연립(다세대)주택,아파트(주상복합아파트의 주거용도 부분 포함)로서 각 호(실)가 모두 주택으로만 쓰이는 것나.상품의 특이사항1)아파트 ,맨션,연립,빌딩,단독주택 등 전용주택 및 가재도구가 화재,벼락,폭발에 의한 사고로 입은 손해를 보상하는 보험입니다.2)화재위험 외에도 도난,풍수재,전기위험 등 다양한 보장 특약을 위험에 맞게 선택하여 보험에 가입할 수 있습니다.2.보험금 지급사유 및 지급제한사항가.보장의 종류 동 보험의 주요 보장은 아래와 같습니다.선택구분보장내용기본계약KB주택화재보험 보통약관선택계약전기위험 특별약관선택계약풍수재위험 특별약관선택계약지진위험 특별약관선택계약특수건물 풍수재위험 특별약관선택계약특수건물 항공기 및 낙하물위험 특별약관선택계약신체손해배상책임 특별약관(특수건물)선택계약확장위험  특별약관(Ⅰ)선택계약확장위험 특별약관(Ⅱ)선택계약붕괴, 침강 및 사태확장 특별약관선택계약소요, 노동쟁의, 항공기 및 차량위험 특별약관선택계약도난위험 특별약관선택계약급배수설비누출손해 특별약관선택계약스프링클러누출손해 특별약관선택계약유리손해 특별약관나.보험금 지급사유기본계약은 화재,폭발 또는 파열로 아래의 손해를 보상합니다.1.사고에 따른 직접 손해2.사고에 따른 소방 손해(화재진압과정에서 발생하는 손해)3.사고에 따른 피난 손해(피난지에서 5일 동안에 보험의 목적에 생긴 위 제1호 및 제2호의 손해를 포함합니다)※용어의 정의폭발,파열이라 함은 급격한 산화반응을 포함하는 파괴 또는 그 현상을 말합니다.선택계약별 보장 내용은 해당 특별약관을 참고하시기 바랍니다.다.보험금 지급 제한 사항보험계약자나 피보험자 또는 이들의 법정대리인의 고의 또는 중대한 과실 등,이 보험 보통약관,특별약관에서 정한 보상하지 않는 손해에 해당되는 경우에는 회사는 책임을지지 않습니

# PDF 문서 청킹

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 청크 방식 정의
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)

final_chunks = []
for index, document in enumerate(documents):
    # 문서 청킹
    text_chunks = text_splitter.split_text(document)
    final_chunks.extend(text_chunks)
    # 청크 문서 개수 출력
    print(f"문서 {index} 청크 개수: {len(text_chunks)}")

# 전체 청크 개수 출력
print(f"전체 청크 개수: {len(final_chunks)}")
# 청크 출력
print(final_chunks[0])
print(final_chunks[1])

문서 0 청크 개수: 6
문서 1 청크 개수: 35
문서 2 청크 개수: 11
전체 청크 개수: 52
KB주택화재보험 상품요약서1.가입자격 제한 등 상품의 특이사항가.가입자격 제한주택으로만 쓰이는 건물 중 다음의 것 및 그 수용가재에 대하여 적용합니다.1)단독주택2)주택의 부속건물로서 가재만을 수용하는데 쓰이는 것3)연립(다세대)주택,아파트(주상복합아파트의 주거용도 부분 포함)로서 각 호(실)가 모두 주택으로만 쓰이는 것나.상품의 특이사항1)아파트 ,맨션,연립,빌딩,단독주택 등 전용주택 및 가재도구가 화재,벼락,폭발에 의한 사고로 입은 손해를 보상하는 보험입니다.2)화재위험 외에도 도난,풍수재,전기위험 등 다양한 보장 특약을 위험에 맞게 선택하여 보험에 가입할 수 있습니다.2.보험금 지급사유 및 지급제한사항가.보장의 종류 동 보험의 주요 보장은 아래와 같습니다.선택구분보장내용기본계약KB주택화재보험 보통약관선택계약전기위험 특별약관선택계약풍수재위험 특별약관선택계약지진위험 특별약관선택계약특수건물 풍수재위험 특별약관선택계약특수건물 항공기 및 낙하물위험 특별약관선택계약신체손해배상책임
특별약관선택계약풍수재위험 특별약관선택계약지진위험 특별약관선택계약특수건물 풍수재위험 특별약관선택계약특수건물 항공기 및 낙하물위험 특별약관선택계약신체손해배상책임 특별약관(특수건물)선택계약확장위험  특별약관(Ⅰ)선택계약확장위험 특별약관(Ⅱ)선택계약붕괴, 침강 및 사태확장 특별약관선택계약소요, 노동쟁의, 항공기 및 차량위험 특별약관선택계약도난위험 특별약관선택계약급배수설비누출손해 특별약관선택계약스프링클러누출손해 특별약관선택계약유리손해 특별약관나.보험금 지급사유기본계약은 화재,폭발 또는 파열로 아래의 손해를 보상합니다.1.사고에 따른 직접 손해2.사고에 따른 소방 손해(화재진압과정에서 발생하는 손해)3.사고에 따른 피난 손해(피난지에서 5일 동안에 보험의 목적에 생긴 위 제1호 및 제2호의 손해를 포함합니다)※용어의 정의폭발,파열이라 함은 급격한 산화반응을 포함하는 파괴 또는 그 현상을 말합니다.선택

# 청크 임베딩

In [30]:
from pinecone import Pinecone

# Pinecone 클라이언트 초기화
pc = Pinecone(api_key=PINECONE_API_KEY)

# 텍스트 데이터를 Pinecone이 색인할 수 있는 형태의 벡터로 변환
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=final_chunks,
    parameters={"input_type": "passage", "truncate": "END"}
)

print(embeddings)

EmbeddingsList(
  model='multilingual-e5-large',
  vector_type='dense',
  data=[
    {'vector_type': dense, 'values': [0.01428985595703125, 0.0012063980102539062, ..., -0.043487548828125, 0.01421356201171875]},
    {'vector_type': dense, 'values': [0.00832366943359375, -0.002899169921875, ..., -0.03466796875, 0.0010328292846679688]},
    ... (48 more embeddings) ...,
    {'vector_type': dense, 'values': [-0.0118408203125, -0.0160064697265625, ..., -0.0246429443359375, -0.01232147216796875]},
    {'vector_type': dense, 'values': [0.0155487060546875, 0.00150299072265625, ..., -0.0311126708984375, -0.00859832763671875]}
  ],
  usage={'total_tokens': 12410}
)


# 인덱스 생성

In [31]:
from pinecone import ServerlessSpec

# 보험 문서 인덱스 생성
index_name = "insurance"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024, # 임베딩 차원
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# 문서 저장

In [32]:
# 벡터를 저장할 인덱스 가져오기
index = pc.Index(index_name)

# 인덱스에 저장하기위해 records 생성
# id: 필수 값, values: 임베딩 벡터, metadata: 텍스트
records = []
for i, (d, e) in enumerate(zip(final_chunks, embeddings.data)):
    records.append({
        "id": str(i),
        "values": e['values'],
        "metadata": {'text': d}
    })

# records를 index에 저장하기
index.upsert(
    vectors=records,
    namespace="insurance-namespace"
)

{'upserted_count': 52}

# 후보군 문서 검색

In [33]:
index = pc.Index("insurance")

query = "의무보험 가입대상 자동차가 뭐야?"

x = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

results = index.query(
    namespace="insurance-namespace",
    vector=x[0].values, # 입력 쿼리의 벡터
    top_k=10, # 유사도 점수 기준으로 상위 10개 검색
    include_values=False, # 응답에 벡터 값을 포함하지 않는다
    include_metadata=True # 응답에 메타 데이터를 포함한다
)

print(results)

{'matches': [{'id': '11',
              'metadata': {'text': '보험가입금액 초과 손해 :           1사고당 '
                                   '5,000만원                        ④ 의무보험 가입대상 '
                                   '자동차   ㅇ 자동차관리법 제3조 규정에 의하여 등록된 자동차   ㅇ '
                                   '건설기계관리법 제3조 규정에 의하여 등록된 건설기계중 자배법시행령 제2조에 '
                                   '정한 '
                                   '건설기계＊＊덤프트럭,트럭적재식콘크리트펌프,타이어식기중기,트럭적재식아스팔트살포기,콘크리트믹서트럭,타이어식굴삭기,「건설기계관리법시행령」별표1제26호에따른특수건설기계중트럭지게차,도로보수트럭,노면측정장비(노면측정장치를가진자주식)  '
                                   '⑤ 의무보험 미가입시 제재   ㅇ 미가입시 과태료 '
                                   '의무보험미가입기간자가용자동차사업용자동차이륜자동차대인Ⅰ대물대인Ⅰ대물대인Ⅰ대물10일이내10,000원5,000원30,000원5,000원6,000원3,000원매1일 '
                                   '초과4,000원2,000원8,000원2,000원1,200원600원최고한도60만원30만원100만원30만원20만원10만원        '
                                   '＊자배법 시행령 [별표 5]   ㅇ 미가입운행시'},
              'score': 0.873087227,
              'values': []},
             {'id': '8',
     

# 최종 문서 필터링 - Reranking

In [34]:
result = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=query,
    documents=[{"id": match["id"], "text": match["metadata"]["text"]} for match in results["matches"]],
    top_n=3,
    return_documents=True,
    parameters={
        "truncate": "END"
    }
)

print(result)

RerankResult(
  model='bge-reranker-v2-m3',
  data=[{
    index=0,
    score=0.9639839,
    document={
        id='11',
        text='보험가입금액 초과 손해 :           1사고당 5,000만원                        ④ 의무보험 가입대상 자동차   ㅇ 자동차관리법 제3조 규정에 의하여 등록된 자동차   ㅇ 건설기계관리법 제3조 규정에 의하여 등록된 건설기계중 자배법시행령 제2조에 정한 건설기계＊＊덤프트럭,트럭적재식콘크리트펌프,타이어식기중기,트럭적재식아스팔트살포기,콘크리트믹서트럭,타이어식굴삭기,「건설기계관리법시행령」별표1제26호에따른특수건설기계중트럭지게차,도로보수트럭,노면측정장비(노면측정장치를가진자주식)  ⑤ 의무보험 미가입시 제재   ㅇ 미가입시 과태료 의무보험미가입기간자가용자동차사업용자동차이륜자동차대인Ⅰ대물대인Ⅰ대물대인Ⅰ대물10일이내10,000원5,000원30,000원5,000원6,000원3,000원매1일 초과4,000원2,000원8,000원2,000원1,200원600원최고한도60만원30만원100만원30만원20만원10만원        ＊자배법 시행령 [별표 5]   ㅇ 미가입운행시'
    }
  },{
    index=2,
    score=0.71012604,
    document={
        id='7',
        text='비사업용 자동차 중 개인소유 자동차KB뉴비즈니스자동차보험개인소유 자가용승용차를 제외한 모든  비사업용 자동차 중 법인소유 자동차영업용君KB영업용자동차보험모든 영업용자동차 및 건설기계KB다이렉트영업용자동차보험KB다이렉트(인터넷)영업용자동차보험이륜차君KB이륜자동차보험이륜자동차 및 원동기장치자전거KB다이렉트이륜자동차보험KB다이렉트(인터넷)이륜자동차보험기타KB운전자보험, KB자동차취급업자종합보험 등 □ 자동차보험 공동인수 제도  ① 대상 : 사고가 발생할 위험이 높거나 또는 기존 가입기간 동안 

---